In [1]:
import pandas as pd
import numpy as np
import sklearn

In [2]:
df = pd.read_csv('Dataframe.csv')
df.head()

,Unnamed: 0,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,lon,lat
0,0,CERTIFIED,"EVEREST CONSULTING GROUP, INC",Computer Programmers,PROGRAMMER ANALYST,Y,55245.0,2012.0,"EDISON, NEW JERSEY",-74.412095,40.518715
1,1,CERTIFIED,SCHULTE ROTH & ZABEL LLP,Computer Systems Analysts,LEGAL PRACTICE PROJECT MANAGER,Y,74422.0,2012.0,"NEW YORK, NEW YORK",-74.005941,40.712784
2,2,CERTIFIED,ERNST &AMP; YOUNG U.S. LLP,MANAGEMENT ANALYSTS,ADVISORY SENIOR,Y,80496.0,2016.0,"BOSTON, MASSACHUSETTS",-71.058880,42.360082
3,4,CERTIFIED,MERRILL LYNCH,COMPUTER AND INFORMATION SYSTEMS MANAGERS,VP; TECHNOLOGY MANAGER-SYSTEMS ENGINEER,Y,138133.0,2015.0,"NEW YORK, NEW YORK",-74.005941,40.712784
4,5,CERTIFIED,DELOITTE CONSULTING LLP,"Software Developers, Systems Software",CONSULTANT,Y,59259.0,2012.0,"WALNUT CREEK, CALIFORNIA",-122.065182,37.910078


In [3]:
'''
Certified: Employer filed the LCA, which was approved by DOL
Certified Withdrawn: LCA was approved but later withdrawn by employer
Withdrawn: LCA was withdrawn by employer before approval
Denied: LCA was denied by DOL
foi transformado em 0 quando DENIED e 1 quando CERTIFIED, CERTIFIED-WITHDRAWN juntos. WITHDRAWN deletado



Occupational name associated with the SOC_CODE. 
SOC_CODE is the occupational code associated with the job being requested for temporary labor condition, 
as classified by the Standard Occupational Classification (SOC) System.
'''
def dummify(data, column_name):
    df = data.copy()
    df2 = pd.concat([df.drop(column_name, axis=1), pd.get_dummies(data[column_name], prefix=column_name)], axis=1)
    return df2


df = df.drop(["WORKSITE","JOB_TITLE","Unnamed: 0"], axis=1)
df = df.dropna()
df["SOC_NAME"] = df["SOC_NAME"].apply(lambda x: x.upper())

df = df.drop(["EMPLOYER_NAME"], axis=1) #mudar depois

df["FULL_TIME_POSITION"] = df["FULL_TIME_POSITION"].apply(lambda x: 0 if x=="N" else 1)

df = df[df.CASE_STATUS != "WITHDRAWN"]
df["CASE_STATUS"] = df["CASE_STATUS"].apply(lambda x: 1 if (x=="CERTIFIED-WITHDRAWN" or x=="CERTIFIED") else 0)

df = dummify(df, "SOC_NAME")

In [4]:
df.head()

,CASE_STATUS,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,lon,lat,SOC_NAME_ACCOUNTANTS,SOC_NAME_ACCOUNTANTS AND AUDITORS,SOC_NAME_ACTUARIES,SOC_NAME_ADMINISTRATIVE SERVICES MANAGERS,...,SOC_NAME_VETERINARIANS,SOC_NAME_VETERINARY ASSISTANTS AND LABORATORY ANIMAL CARETA,SOC_NAME_VETERINARY TECHNOLOGISTS AND TECHNICIANS,SOC_NAME_VOCATIONAL EDUCATION TEACHERS POSTSECONDARY,"SOC_NAME_VOCATIONAL EDUCATION TEACHERS, POSTSECONDARY",SOC_NAME_WEB ADMINISTRATORS,SOC_NAME_WEB DEVELOPERS,"SOC_NAME_WHOLESALE AND RETAIL BUYERS, EXCEPT FARM PRODUCTS",SOC_NAME_WRITERS AND AUTHORS,SOC_NAME_ZOOLOGISTS AND WILDLIFE BIOLOGISTS
0,1,1,55245.0,2012.0,-74.412095,40.518715,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,74422.0,2012.0,-74.005941,40.712784,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,80496.0,2016.0,-71.058880,42.360082,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,138133.0,2015.0,-74.005941,40.712784,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,59259.0,2012.0,-122.065182,37.910078,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
col = []

for i in df.columns:
    if i.startswith("SOC_NAME_"):
        col.append(i)

X = df[col].values
y = df["CASE_STATUS"].values

index = round(df.index.size*0.8)

X_train, X_test = X[:index], X[index:]
y_train, y_test = y[:index], y[index:]


print(len(X_train),len(y_train))

67001 67001


In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


model = LogisticRegression()
model.fit(X_train, y_train)
print("fitou")
y_pred = model.predict(X_test)
print("predictou")
print("Acurácia: {}".format(accuracy_score(y_test, y_pred)))


C:\Users\Freddy Dratwa\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


fitou
predictou
Acurácia: 0.969134328358209
